# Factory list 

In [19]:
from rapidfuzz import fuzz
import pandas as pd
from itertools import combinations
from collections import defaultdict
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import networkx as nx
import os
import re
import pandas as pd
from collections import defaultdict
from itertools import combinations
from openpyxl import load_workbook
from neo4j import GraphDatabase
from pymongo import MongoClient
import pandas as pd
import os
import pandas as pd
from collections import defaultdict
from itertools import combinations
from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font
from copy import copy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:

# === Connect to MongoDB ===
mongo_client = MongoClient("") #to be added
mongo_db = mongo_client["tuone"] 
articles_collection = mongo_db["articles"] # change "articles"

# === Fetch all documents with _id and meta fields ===
docs = list(
    articles_collection.find(
        {},  # No filter — get all documents
        {
            "_id": 1,
            "meta.ID": 1,
            "meta.url": 1,
            "meta.date": 1
        }
    )
)

# === Create DataFrame ===
data = [{
    "_id": str(doc.get("_id")),
    "ID": doc.get("meta", {}).get("ID"),
    "url": doc.get("meta", {}).get("url"),
    "date": doc.get("meta", {}).get("date")
} for doc in docs]

df_meta = pd.DataFrame(data)

# === Display sample or save ===
print(df_meta.head())
# Optionally save to file:
# df_meta.to_csv("article_meta_export.csv", index=False)


                        _id       ID  \
0  67f52a45981040986eab706d  2700008   
1  67f52a45981040986eab706e  2700013   
2  67f52a45981040986eab706f  2700025   
3  67f52a46981040986eab7070  2700027   
4  67f52a46981040986eab7071  2700032   

                                                 url                 date  
0  https://www.electrive.com/2016/01/18/united-ki...  2016-01-18 00:00:00  
1  https://www.electrive.com/2016/01/26/baic-chin...  2016-01-26 00:00:00  
2  https://www.electrive.com/2016/01/25/bmw-wanxi...  2016-01-25 00:00:00  
3  https://www.electrive.com/2016/01/20/morgan-to...  2016-01-20 00:00:00  
4  https://www.electrive.com/2016/01/29/volkswage...  2016-01-29 00:00:00  


In [34]:
import pandas as pd
from functools import reduce
from collections import defaultdict

# === Helper Functions ===
def extract_ids(x):
    if isinstance(x, list):
        return [i.split("_")[1] if "_" in i else i for i in x]
    return []

def safe_lookup_list(ids, lookup, key):
    if not isinstance(ids, list):
        return []
    result = [lookup.get(i, {}).get(key, None) for i in ids]
    while len(result) < len(ids):
        result.append(None)
    return result[:len(ids)]

def build_reconciliation_lookup(log):
    df_log = pd.DataFrame(log)
    df_log["article_id"] = df_log["original_unique_id"].str.extract(r"(^[^_]+)")
    product_ids = df_log[df_log["entity_type"] == "product"].groupby("new_unique_id")["article_id"].apply(set).to_dict()
    company_ids = df_log[df_log["entity_type"] == "company"].groupby("new_unique_id")["article_id"].apply(set).to_dict()
    jv_ids = df_log[df_log["entity_type"] == "joint_venture"].groupby("new_unique_id")["article_id"].apply(set).to_dict()
    investment_ids = df_log[df_log["entity_type"] == "investment"].groupby("new_unique_id")["article_id"].apply(set).to_dict()
    capacity_ids = df_log[df_log["entity_type"] == "capacity"].groupby("new_unique_id")["article_id"].apply(set).to_dict()

    for uid in df_log["new_unique_id"]:
        fallback = {uid.split("_")[0]}
        if uid.startswith("product_"):
            product_ids.setdefault(uid, fallback)
        elif uid.startswith("company_"):
            company_ids.setdefault(uid, fallback)
        elif uid.startswith("investment_"):
            investment_ids.setdefault(uid, fallback)
        elif uid.startswith("capacity_"):
            capacity_ids.setdefault(uid, fallback)
        elif uid.startswith("joint_venture_"):
            jv_ids.setdefault(uid, fallback)

    return df_log, capacity_ids, product_ids, company_ids, jv_ids, investment_ids

def make_article_id_to_url(df_meta):
    df_meta["ID"] = df_meta["ID"].astype(str)
    return df_meta.set_index("ID")["url"].to_dict()

def resolve_urls(article_ids, article_id_to_url):
    if not isinstance(article_ids, (list, set)):
        return []
    return [article_id_to_url[aid] for aid in article_ids if aid in article_id_to_url]

def resolve_urls_from_uid(uids, article_id_to_url):
    if not isinstance(uids, list):
        return []
    return [article_id_to_url.get(uid[:7]) for uid in uids if isinstance(uid, str) and uid[:7] in article_id_to_url]


def deduplicate_nodes_and_rels(df_nodes, df_rels):
    return (
        df_nodes.drop_duplicates(subset="unique_id"),
        df_rels.drop_duplicates(subset=["source", "target", "type"])
    )

def extract_node_subsets(df_nodes):
    return {
        "joint_ventures": df_nodes[df_nodes["label"].str.lower() == "joint_venture"],
        "factories": df_nodes[df_nodes["label"].str.lower().str.contains("factory", na=False)].copy(),
        "capacities": df_nodes[df_nodes["label"].str.lower() == "capacity"],
        "products": df_nodes[df_nodes["label"].str.lower() == "product"],
        "companies": df_nodes[df_nodes["label"].str.lower() == "company"],
        "investment": df_nodes[df_nodes["label"].str.lower() == "investment"]
    }

def extract_relationship_subsets(df_rels):
    return {
        "owns": df_rels[df_rels["type"].str.lower() == "owns"],
        "at": df_rels[df_rels["type"].str.lower() == "at"],
        "produced_at": df_rels[df_rels["type"].str.lower() == "produced_at"],
        "funds": df_rels[df_rels["type"].str.lower() == "funds"]
    }

def group_linked_nodes(rel_df, source_nodes, source_col, target_col, entity_label_prefix):
    rel_df = rel_df.copy()
    df = rel_df.rename(columns={"source": source_col, "target": target_col})
    df = df.merge(
        source_nodes[["unique_id", "name"]].rename(columns={
            "unique_id": f"{entity_label_prefix}_unique_id",
            "name": f"{entity_label_prefix}_name"
        }),
        left_on=source_col, right_on=f"{entity_label_prefix}_unique_id", how="left"
    )
    grouped = df.groupby(target_col).agg({
        f"{entity_label_prefix}_unique_id": lambda x: list(x.dropna().unique()),
        f"{entity_label_prefix}_name": lambda x: list(x.dropna().unique())
    }).reset_index().rename(columns={target_col: "factory_unique_id"})
    return grouped



def run_factory_centric_enrichment(df_all_nodes, df_all_rels, df_meta):
    # capacity_article_ids, product_article_ids, company_article_ids, joint_venture_article_ids, investment_article_ids = build_reconciliation_lookup(main_reconciliation_log)
    article_id_to_url = make_article_id_to_url(df_meta)
    df_all_nodes, df_all_rels = deduplicate_nodes_and_rels(df_all_nodes, df_all_rels)
    nodes = extract_node_subsets(df_all_nodes)
    rels = extract_relationship_subsets(df_all_rels)

    # Extract city and country from already flattened factory nodes
    df_factory_locations = nodes["factories"][["name", "unique_id", "location_city", "location_country"]].rename(
        columns={
            "unique_id": "factory_unique_id",
            "location_city": "factory_city",
            "location_country": "factory_country",
            "name":"factory_name"
        }
    )


    df_owns_comp = group_linked_nodes(rels["owns"], nodes["companies"], "source", "target", "owner_company")
    df_owns_jv = group_linked_nodes(rels["owns"], nodes["joint_ventures"], "source", "target", "owner_jv")
    df_funds = group_linked_nodes(rels["funds"], nodes["investment"], "source", "target", "investment")
    df_products = group_linked_nodes(rels["produced_at"], nodes["products"], "source", "target", "product")
    df_capacities = group_linked_nodes(rels["at"], nodes["capacities"], "source", "target", "capacity")

    df_master = reduce(lambda left, right: pd.merge(left, right, on="factory_unique_id", how="outer"), [
        df_owns_comp, df_owns_jv, df_funds, df_products, df_capacities
    ])
    # Merge city and country info
    df_master = df_master.merge(df_factory_locations, on="factory_unique_id", how="left")


    inv_lookup = nodes["investment"].set_index("unique_id")[["name", "status", "amount", "phase"]].to_dict("index")
    cap_lookup = nodes["capacities"].set_index("unique_id")[["name", "status", "amount", "phase"]].to_dict("index")

    df_master["investment_name"] = df_master["investment_unique_id"].apply(lambda uids: safe_lookup_list(uids, inv_lookup, "name"))
    df_master["investment_status"] = df_master["investment_unique_id"].apply(lambda uids: safe_lookup_list(uids, inv_lookup, "status"))
    df_master["investment_amount"] = df_master["investment_unique_id"].apply(lambda uids: safe_lookup_list(uids, inv_lookup, "amount"))
    df_master["investment_phase"] = df_master["investment_unique_id"].apply(lambda uids: safe_lookup_list(uids, inv_lookup, "phase"))

    df_master["capacity_name"] = df_master["capacity_unique_id"].apply(lambda ids: safe_lookup_list(ids, cap_lookup, "name"))
    df_master["capacity_status"] = df_master["capacity_unique_id"].apply(lambda ids: safe_lookup_list(ids, cap_lookup, "status"))
    df_master["capacity_amount"] = df_master["capacity_unique_id"].apply(lambda ids: safe_lookup_list(ids, cap_lookup, "amount"))
    df_master["capacity_phase"] = df_master["capacity_unique_id"].apply(lambda ids: safe_lookup_list(ids, cap_lookup, "phase"))


    df_master["factory_urls"] = df_master["factory_unique_id"].apply(lambda uids: resolve_urls_from_uid(uids, article_id_to_url))
    df_master["product_urls"] = df_master["product_unique_id"].apply(lambda uids: resolve_urls_from_uid(uids, article_id_to_url))
    df_master["capacity_urls"] = df_master["capacity_unique_id"].apply(lambda uids: resolve_urls_from_uid(uids, article_id_to_url))
    df_master["owner_company_urls"] = df_master["owner_company_unique_id"].apply(lambda uids: resolve_urls_from_uid(uids, article_id_to_url))
    df_master["owner_jv_urls"] = df_master["owner_jv_unique_id"].apply(lambda uids: resolve_urls_from_uid(uids, article_id_to_url))
    df_master["investment_urls"] = df_master["investment_unique_id"].apply(lambda uids: resolve_urls_from_uid(uids, article_id_to_url))

    # df_master["factory_article_ids"] = df_master["factory_unique_id"].apply(extract_ids)
    # df_master["factory_urls"] = df_master["factory_article_ids"].apply(lambda ids: resolve_urls(ids, article_id_to_url))

    # df_master["product_article_ids"] = df_master["product_unique_id"].apply(extract_ids)
    # df_master["product_urls"] = df_master["product_article_ids"].apply(lambda ids: resolve_urls(ids, article_id_to_url))

    # df_master["capacity_article_ids"] = df_master["capacity_unique_id"].apply(extract_ids)
    # df_master["capacity_urls"] = df_master["capacity_article_ids"].apply(lambda ids: resolve_urls(ids, article_id_to_url))

    # df_master["company_article_ids"] = df_master["owner_company_unique_id"].apply(
    #     lambda uids: [aid for uid in uids for aid in company_article_ids.get(uid, [])] if isinstance(uids, list) else [])
    # df_master["company_urls"] = df_master["company_article_ids"].apply(lambda ids: resolve_urls(ids, article_id_to_url))

    # df_master["joint_venture_article_ids"] = df_master["owner_jv_unique_id"].apply(
    #     lambda uids: [aid for uid in uids for aid in joint_venture_article_ids.get(uid, [])] if isinstance(uids, list) else [])
    # df_master["joint_venture_urls"] = df_master["joint_venture_article_ids"].apply(lambda ids: resolve_urls(ids, article_id_to_url))

    df_master_final = df_master[[
        "factory_name", "factory_country", "factory_city", "factory_urls",
        "owner_company_name", 
        "owner_jv_name", 
        "product_name", 
        "capacity_name", "capacity_status",  "capacity_phase", "capacity_amount",
        "investment_name", "investment_status", "investment_phase", "investment_amount", 
    ]]


    # df_canonical = pd.DataFrame([{"entity_type": k, "unique_id": v} for k, vs in canonical_entities.items() for v in vs])

    df_factories_pivot = df_master.explode(["factory_unique_id"])
    df_owner_companies_pivot = df_master.explode("owner_company_name")
    df_owner_jvs_pivot = df_master.explode("owner_jv_name")
    df_products_pivot = df_master.explode(["product_name"])
    df_capacities_pivot = df_master.explode(["capacity_name", "capacity_status", "capacity_amount", "capacity_phase"])
    df_investments_pivot = df_master.explode(["investment_name", "investment_status", "investment_amount", "investment_phase"])

    with pd.ExcelWriter("reconciliation_outputs_factory.xlsx", engine="openpyxl") as writer:
        df_master.to_excel(writer, sheet_name="factory", index=False)
        df_master_final.to_excel(writer, sheet_name="summary_view_factory", index=False)
        df_factories_pivot.to_excel(writer, sheet_name="pivot_factories", index=False)
        df_owner_companies_pivot.to_excel(writer, sheet_name="pivot_owner_companies", index=False)
        df_owner_jvs_pivot.to_excel(writer, sheet_name="pivot_owner_jvs", index=False)
        df_products_pivot.to_excel(writer, sheet_name="pivot_products", index=False)
        df_capacities_pivot.to_excel(writer, sheet_name="pivot_capacities", index=False)
        df_investments_pivot.to_excel(writer, sheet_name="pivot_investments", index=False)
        # df_reconcile_log.to_excel(writer, sheet_name="reconciliation_log_factory", index=False)
        # df_canonical.to_excel(writer, sheet_name="canonical_entities_factory", index=False)

    print("\u2705 Saved factory-centric outputs to reconciliation_outputs_factory.xlsx")

# === Sample Data ===
# Run enrichment
run_factory_centric_enrichment(df_all_nodes, df_all_rels, df_meta)


✅ Saved factory-centric outputs to reconciliation_outputs_factory.xlsx
